In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
import csv
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame

df= pd.read_csv(r'C:\Users\dkrenn\Desktop\GW-Bootcamp-Final-Project\Data Analysis\Resources\cleaned_data.csv')
df.head()

,Unnamed: 0,ID,AreaName,VicAge,VicSex,VicRace,StatusCode,Lat,Lon,CrimeType,AgeGroup,AddressType,Arrest
0,0,190326475,Wilshire,999,M,Multiracial,AA,34.0375,-118.3506,Theft,Unknown,Public Space,Yes
1,1,200106753,Central,47,M,Multiracial,IC,34.0444,-118.2628,Theft,40-54,Public Space,No
2,2,200320258,Southwest,19,NaN,,IC,34.0210,-118.3002,Theft,Young Adult,Residential,No
3,3,200907217,Van Nuys,19,M,Multiracial,IC,34.1576,-118.4387,Theft,Young Adult,Buisness,No
4,4,220614831,Hollywood,28,M,Hispanic,IC,34.0944,-118.3277,Theft,25-39,Public Space,No


In [3]:
# Remove Unneeded Columns
df= df.drop(columns =["Unnamed: 0", "ID", "StatusCode", "Lat", "Lon", "AgeGroup"])
df.head()

,AreaName,VicAge,VicSex,VicRace,CrimeType,AddressType,Arrest
0,Wilshire,999,M,Multiracial,Theft,Public Space,Yes
1,Central,47,M,Multiracial,Theft,Public Space,No
2,Southwest,19,NaN,,Theft,Residential,No
3,Van Nuys,19,M,Multiracial,Theft,Buisness,No
4,Hollywood,28,M,Hispanic,Theft,Public Space,No


In [4]:
# Remove Missing Age Codes
df=df.loc[df["VicAge"]<999]


In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
X_encoded = pd.get_dummies(df, dtype=int)
X_encoded


,VicAge,AreaName_77th Street,AreaName_Central,AreaName_Devonshire,AreaName_Foothill,AreaName_Harbor,AreaName_Hollenbeck,AreaName_Hollywood,AreaName_Mission,AreaName_N Hollywood,...,CrimeType_Sex Crime,CrimeType_Theft,CrimeType_Weapons,AddressType_Buisness,AddressType_Other,AddressType_Public Space,AddressType_Public Transit,AddressType_Residential,Arrest_No,Arrest_Yes
1,47,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
2,19,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
3,19,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
4,28,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
5,41,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883982,29,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
883983,30,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
883984,24,0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
883985,53,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [9]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


# Split our preprocessed data into our features and target arrays
y = X_encoded["Arrest_Yes"]
X = X_encoded.drop(columns="Arrest_Yes")
X = X.drop(columns="Arrest_No")
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model1 = tf.keras.models.Sequential()

# First hidden layer
nn_model1.add(tf.keras.layers.Dense(units=6, activation="relu", input_dim=47))

# Second hidden layer
nn_model1.add(tf.keras.layers.Dense(units=6, activation="relu"))

# Output layer
nn_model1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 6)                 288       
                                                                 
 dense_4 (Dense)             (None, 6)                 42        
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 337 (1.32 KB)
Trainable params: 337 (1.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn_model1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn_model1.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
15529/15529 [==============================] - 21s 1ms/step - loss: 0.2840 - accuracy: 0.9065
Epoch 2/50
15529/15529 [==============================] - 20s 1ms/step - loss: 0.2799 - accuracy: 0.9066
Epoch 3/50
15529/15529 [==============================] - 21s 1ms/step - loss: 0.2794 - accuracy: 0.9066
Epoch 4/50
15529/15529 [==============================] - 20s 1ms/step - loss: 0.2791 - accuracy: 0.9066
Epoch 5/50
15529/15529 [==============================] - 19s 1ms/step - loss: 0.2790 - accuracy: 0.9066
Epoch 6/50
15529/15529 [==============================] - 19s 1ms/step - loss: 0.2788 - accuracy: 0.9066
Epoch 7/50
15529/15529 [==============================] - 20s 1ms/step - loss: 0.2787 - accuracy: 0.9066
Epoch 8/50
15529/15529 [==============================] - 20s 1ms/step - loss: 0.2787 - accuracy: 0.9066
Epoch 9/50
15529/15529 [==============================] - 20s 1ms/step - loss: 0.2786 - accuracy: 0.9066
Epoch 10/50
15529/15529 [==============================

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5177/5177 - 5s - loss: 0.2803 - accuracy: 0.9057 - 5s/epoch - 874us/step
Loss: 0.2803204655647278, Accuracy: 0.905684769153595


In [19]:
# Second Attempt- Neurons added to layers
nn_model2 = tf.keras.models.Sequential()

# First hidden layer
nn_model2.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=47))

# Second hidden layer
nn_model2.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model2.summary()

# Compile the model
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model2.fit(X_train_scaled, y_train, epochs=50)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 16)                768       
                                                                 
 dense_10 (Dense)            (None, 16)                272       
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 1057 (4.13 KB)
Trainable params: 1057 (4.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
15529/15529 [==============================] - 22s 1ms/step - loss: 0.2820 - accuracy: 0.9067
Epoch 2/50
15529/15529 [==============================] - 27s 2ms/step - loss: 0.2789 - accuracy: 0.9068
Epoch 3/50
15529/15529 [==============================] - 21s 1ms/step - los

In [20]:
# Second Evaluation
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5177/5177 - 7s - loss: 0.2797 - accuracy: 0.9060 - 7s/epoch - 1ms/step
Loss: 0.27969083189964294, Accuracy: 0.906046986579895


In [21]:
# Export our model to HDF5 file
nn_model2.save("Saved NN Models/ArrestRateModel.h5")

C:\Users\dkrenn\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
